# *Retrieval Augmented Generation* con las conferencias mañaneras

## Una pregunta muy específica

Antes de comenzar, quiero mostrarles cuál es la utilidad de utilizar una técnica como esta, comenzaré haciéndole una pregunta muy específica a un modelo genérico:

In [2]:
import os

In [51]:
from openai import OpenAI

client = OpenAI()

def query_llm(prompt, model="gpt-4o-turbo"):
    completions = client.chat.completions.create(

    model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=0.0,
    )

    return completions.choices[0].message.content

In [4]:
pregunta = "¿Qué significa ser aspiracionista?"

prompt = """Eres Andrés Manuel Lopez Obrador, presidente de México.
Responde a la pregunta como si la respuesta la estuviera dando Andrés Manuel Lopez Obrador.

Pregunta: {question}
"""
final_prompt = prompt.format(question=pregunta)

respuesta = query_llm(final_prompt)
print(respuesta)

Ser aspiracionista significa tener la convicción de que es posible lograr un cambio positivo en nuestro país, trabajando juntos para construir un México más justo, equitativo y próspero para todos. Es tener la esperanza y la determinación de alcanzar nuestras metas y sueños, siempre con honestidad, transparencia y amor por México. ¡Sí se puede!


## Descargando los datos de mi otro repositorio

In [5]:
import tempfile
from pathlib import Path
import subprocess

temporary_directory = tempfile.mkdtemp()

In [6]:

conferencias_repo_url = "https://github.com/fferegrino/mananeras.git"
conferencias_repo_dir = Path(temporary_directory, "mananeras")

subprocess.run(["git", "clone", "-q", "--single-branch","--branch", 'embeddings-tutorial', "--depth", "1", conferencias_repo_url, str(conferencias_repo_dir)])

Note: switching to '4e4283ee9fd3b07eee7574cf2b6200177770a35f'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



CompletedProcess(args=['git', 'clone', '-q', '--single-branch', '--branch', 'embeddings-tutorial', '--depth', '1', 'https://github.com/fferegrino/mananeras.git', '/var/folders/f2/32fx7hzs4qb3mtt5x1kt75bc0000gn/T/tmpyl0kxye5/mananeras'], returncode=0)

## Cargando los documentos

Todo sistema *RAG* comienza cargando un conjunto inicial de documentos. 

In [10]:
import mananeras

conferencias = mananeras.lee_todas(conferencias_repo_dir)

In [11]:
len(conferencias)

2674

In [12]:
print(conferencias[1].titulo)
print(conferencias[1].fecha)
print(conferencias[1].participaciones[6])

Versión estenográfica. Conferencia de prensa del presidente Andrés Manuel López Obrador del 14 de noviembre de 2022
2022-11-14
Participacion(hablante='PREGUNTA', dialogos=['Gracias, presidente. Dalila Escobar, de Proceso.', 'Bueno, porque es un asunto coyuntural, por supuesto, el tema de la manifestación que se llevó a cabo ayer. Solamente un orador, como se había adelantado, que llamó a que no se regresara a una etapa que ya se había superado en torno a dejar que de nuevo el INE caiga en manos de decisiones que tengan que ver solamente con gobierno, esto porque parte de las interpretaciones tienen que ver con que, si es el pueblo el que elige a consejeros, etcétera, sería en todo caso una especie de elección desde quienes estén en el poder.', 'Preguntarle sus impresiones, sobre todo también con la cantidad de personas que asistieron porque, bueno, se manejaba que son entre 10 mil, más o menos, algo así, la Ciudad de México manejó un número un tato reducido a lo que se observó.', '¿Cóm

## Filtrar solo las participaciones del presidente

In [13]:
dialogos_presidente = []

for conferencia in conferencias:
    dialogos_conferencia = []
    for participacion in conferencia.participaciones:
        hablante = participacion.hablante.lower()
        dialogos_participacion = []
        if 'andrés manuel' in hablante or 'andrésmanuel' in hablante:
            for dialogo in participacion.dialogos:
                dialogos_participacion.append(dialogo)
        if len(dialogos_participacion) > 0:
            dialogos_conferencia.append("\n".join(dialogos_participacion))
    if len(dialogos_conferencia) > 0:
        conferencia = {
            "title": conferencia.titulo,
            "document": "\n".join(dialogos_conferencia)
        }
        dialogos_presidente.append(conferencia)

In [14]:
len(dialogos_presidente)

1985

In [15]:
print(dialogos_presidente[0]['document'][:500])

Muy buenos días. Ánimo.
Bueno, vamos, como todos los lunes, a dar a conocer el Quién es quién en los precios.
Y también, se va a hacer una invitación, una convocatoria, un llamamiento porque inicia el Buen Fin de este año 2022. Va a intervenir Ricardo Sheffield y también va a intervenir José Héctor Tejada Shaar, que es el presidente de la Concanaco, y la secretaria de Economía, Raquel Buenrostro; nos acompaña también Andrea Hernández Xoxotla, es administradora general de Servicios al Contribuyen


## Divide documento en partes (*chunks*)

Si estás trabajando con documentos grandes, es importarte dividirlo en partes, que vamos a llamar *chunks*.

Esto cumple dos funciones:

 * Mejorar la relevancia semántica de nuestros embeddings: un documento muy grande puede cubrir demasiados temas, mientras que uno pequeño puede estar más enfocado en un solo tópico
 * Facilitar la tarea del modelo de lenuaje generativo – *chunks* más pequeños hacen que la ventana de contexto sea más pequeña

El proceso de división tiene varios parámetros: el tamaño del *chunk* y el tamaño de traslape entre *chunks*.

Existen diversas técnicas de división de documentos, algunas más complejas que otras, la función que estoy usando debajo es una de las más fáciles pero menos recomendables.

In [16]:
def chunk_document_content(document_content, max_size=300, overlap=10):
    tokens = document_content.split()
    chunks = []
    token_count = len(tokens)
    i = 0
    while i < token_count:
        chunks.append(" ".join(tokens[i:i+max_size]))
        i += max_size - overlap
    return chunks

In [17]:
speech = "Nosotros elegimos ir a la luna. Elegimos ir a la luna. Elegimos ir a la luna en esta década y hacer otras cosas no porque sean fáciles, sino porque son difíciles. Porque esa meta servirá para organizar y medir lo mejor de nuestras energías y habilidades, porque ese desafío es uno que estamos dispuestos a aceptar."

chunks = chunk_document_content(speech, max_size=15, overlap=5)
for idx, chunk in enumerate(chunks):
    print(f"Chunk {idx}: {chunk}\n")

Chunk 0: Nosotros elegimos ir a la luna. Elegimos ir a la luna. Elegimos ir a la

Chunk 1: luna. Elegimos ir a la luna en esta década y hacer otras cosas no porque

Chunk 2: hacer otras cosas no porque sean fáciles, sino porque son difíciles. Porque esa meta servirá

Chunk 3: difíciles. Porque esa meta servirá para organizar y medir lo mejor de nuestras energías y

Chunk 4: mejor de nuestras energías y habilidades, porque ese desafío es uno que estamos dispuestos a

Chunk 5: uno que estamos dispuestos a aceptar.



## Dividiendo las conferencias mientras mantenemos la referencia a la fuente original

In [18]:
chunked_documents = []
chunk_unique_id = 0
for doc_id, document in enumerate(dialogos_presidente):
    chunks = chunk_document_content(document["document"])
    for chunk_id, chunk in enumerate(chunks):
        chunked_documents.append({
            "id": chunk_unique_id,
            "document_id": doc_id,
            "text": chunk
        })
        chunk_unique_id += 1

In [19]:
print(len(chunked_documents))

29174


In [56]:
chunked_documents[1]

{'id': 1,
 'document_id': 0,
 'text': 'los que se manifestaron ayer lo hicieron en contra de la transformación que se está llevando en el país, lo hicieron a favor de los privilegios que ellos tenían antes del gobierno que represento, lo hicieron a favor de la corrupción, lo hicieron a favor del racismo, a favor del clasismo, de la discriminación, ese es el fondo, porque ni modo que Madrazo y Elba Esther y Fox sean demócratas, ¿no? El mismo Woldenberg, que convalidó fraudes electorales cuando estuvo en el INE. Le voy más a la maestra Elba Esther, porque esa no se da baños de pureza, es más sincera que Woldenberg. Porque lo que más molesta es la hipocresía, la simulación. Yo creo que fue muy importante la marcha de ayer, fue como una especie de estriptis político, público, del conservadurismo en México. Y esto es muy bueno, pero muy bueno, porque, si no emerge esto, se mantiene soterrado, y hace mucho daño para tener una sociedad mejor, más justa, más igualitaria, más fraterna. Errores 

## Calculando embeddings

Para generar los embeddings vamos a utilizar un modelo local, descargado de Hugging Face.

In [29]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

/Users/cristhiancuenca/SourceCode/AI/LLMs/mananeras-rag/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [27]:
def get_embedding(text):
    return model.encode(text)

In [53]:
embedding = get_embedding("Hola mundo")
print(embedding.shape)
print(embedding)

(768,)
[ 1.76741742e-02  1.25786006e-01 -1.10591017e-02  8.07093158e-02
  2.19953090e-01  6.60998141e-03 -1.46027803e-02  3.07636335e-02
  6.89360946e-02  2.32642479e-02  5.49176820e-02 -8.51896852e-02
 -5.23980781e-02 -3.60355154e-02  2.28620514e-01 -2.54882693e-01
  1.25053391e-01 -6.04240671e-02 -2.73311496e-01 -4.15843092e-02
  1.15275949e-01 -3.05613689e-02  1.12574160e-01 -9.22448263e-02
  4.08078432e-02  1.07518509e-01 -8.22311491e-02  1.24371707e-01
  6.01274297e-02  7.04638660e-02  1.23964369e-01  1.14023443e-02
  7.12504685e-02 -7.73435365e-03  1.13680713e-01 -8.08565542e-02
  7.96617717e-02 -1.69565957e-02 -1.37967527e-01 -1.77947134e-02
  1.12821549e-01 -9.75775197e-02 -1.08611554e-01  1.07095458e-01
 -1.71698406e-01 -7.62376264e-02 -5.71165830e-02 -1.04472242e-01
  5.31469006e-03  1.16451114e-01  7.42642209e-02  3.84854496e-01
  7.06795380e-02  1.00253969e-01  1.59230560e-01 -6.28339499e-02
  8.33500624e-02 -6.45243824e-02  2.79638004e-02 -2.72255391e-04
  1.06029622e-01 -

## Insertando embeddings en la base de datos vectorial

In [21]:
from annoy import AnnoyIndex
import os

embedding_size = 768
index_name = "full-index-mananeras.ann"

In [22]:
chunk_id_mapping = {}
for chunk in chunked_documents:
    chunk_id_mapping[chunk["id"]] = chunk

In [57]:
if not os.path.exists(index_name):
    index = AnnoyIndex(embedding_size, 'angular')

    for idx, chunk in chunk_id_mapping.items():

        v = get_embedding(chunk["text"])

        index.add_item(idx, v)

    index.build(10)
    index.save(index_name)

## Ejecutando queries

In [24]:
index = AnnoyIndex(embedding_size, 'angular')
index.load(index_name)

True

In [43]:
pregunta = "¿Qué significa ser aspiracionista?"

In [44]:
print(pregunta)
embedding_pregunta = get_embedding(pregunta)
# print(embedding_pregunta)

¿Qué significa ser aspiracionista?


In [45]:
ids_potenciales_respuestas = index.get_nns_by_vector(embedding_pregunta, 5)
ids_potenciales_respuestas

[27980, 12746, 12620, 14628, 20336]

In [46]:
potenciales_respuestas = [chunk_id_mapping[idx] for idx in ids_potenciales_respuestas]
texto_potencial = [chunk["text"] for chunk in potenciales_respuestas]
texto_potencial

['en el habla de los pueblos hay distintos modismos, lo que más se entiende es ‘mordida’, así, más general. A veces los escritores presumidos dicen: ‘Bájale’, nos están escribiendo. No, es ‘súbele’, o sea, para que te entiendan; si no, los que comprenden lo que estás escribiendo son muy pocos. Hay que utilizar el buen castellano, evitar los tecnicismos, no hablar físico, como se decía antes, porque se piensa que eso da caché si se utilizan palabras rebuscadas. Bueno, pues resulta que al señor García Luna, jefe de la seguridad durante el gobierno del Calderón, se le acusa en Estados Unidos de estar recibiendo ‘mordidas’ o de haber recibido ‘mordidas’, dinero para darle protección al grupo o al cártel de Guzmán Loera, él los protegía y a cambio de esa protección le daban dinero, se le acusa que desde el gobierno se perseguía a otras bandas mientras se protegía a la de Guzmán Loera. También en ese entonces se permitió que en secreto se introdujeran armas de Estados Unidos a México, supues

## Usando una LLM para generar respuestas

In [41]:
def get_answer(question, context_documents):
    context_divider = "\n---\n"
    question = question.strip()

    context = context_divider.join(context_documents)

    template_rag = """Eres Andrés Manuel Lopez Obrador, presidente de México.
Responde a la pregunta basándote en el contexto de lo dicho por el presidente.
El contexto está delimitado por las comillas invertidas.
Contesta como si la respuesta la estuviera dando Andrés Manuel Lopez Obrador.

```
{context}
```

Pregunta: {question}
"""
    final_prompt = template_rag.format(context=context, question=question)
    return query_llm(final_prompt)


In [52]:
print(pregunta)
print()
print(get_answer(pregunta, texto_potencial))

¿Qué significa ser aspiracionista?

Ser aspiracionista significa tener una actitud de querer triunfar a toda costa, salir adelante y alcanzar el éxito personal, sin importar los medios utilizados. También implica una mentalidad egoísta y enfocada en el propio beneficio, sin considerar necesariamente el bienestar de los demás.


In [78]:
print("Fuentes:")
documentos_originales = [dialogos_presidente[chunk["document_id"]]["title"] for chunk in potenciales_respuestas]
print("\n".join(documentos_originales))

Fuentes:
Versión estenográfica. Conferencia de prensa del presidente Andrés Manuel López Obrador del 1 de octubre de 2020
Versión estenográfica. Conferencia de prensa del presidente Andrés Manuel López Obrador del 19 de abril de 2022
Versión estenográfica. Conferencia de prensa del presidente Andrés Manuel López Obrador del 4 de julio de 2023
Versión estenográfica. Conferencia de prensa del presidente Andrés Manuel López Obrador del 5 de octubre de 2021
Versión estenográfica. Conferencia de prensa del presidente Andrés Manuel López Obrador del 21 de octubre de 2021


In [35]:
print(pregunta)
print()
prompt = """Eres Andrés Manuel Lopez Obrador, presidente de México.
Responde a la pregunta como si la respuesta la estuviera dando Andrés Manuel Lopez Obrador.

Pregunta: {question}
"""
final_prompt = prompt.format(question=pregunta)
print(query_llm(final_prompt))

¿que proyectos hay donde este relacionado el litio?

Como presidente de México, estamos trabajando en varios proyectos relacionados con el litio, un recurso estratégico para el desarrollo de tecnologías limpias y renovables. Estamos explorando la posibilidad de extraer litio de nuestras reservas naturales y desarrollar una industria nacional que nos permita aprovechar este recurso de manera sustentable. Además, estamos buscando alianzas con otros países para impulsar proyectos conjuntos que beneficien a nuestra nación y al mundo entero. El litio es clave en la transición hacia una economía verde y estamos comprometidos en aprovecharlo de manera responsable y en beneficio de todos los mexicanos.
